Using the artists schema, this script iterates over artistID's and gets the corresponding list of songID's, then appends each resulting (artistID,songID) pair to the song_ids.txt file.

In [39]:
# functions _get and get_artist_songs() in this code 
# are courtesy of GitHub user imdkm:
# https://gist.github.com/imdkm/a60247b59ff1881fa4bb8846a9b44c96

In [40]:
import requests, json
from time import sleep

In [41]:
# Secret token
token = open("ACCESS_TOKEN.txt", "r").read()

In [42]:
# constant values.
BASE_URL = "https://api.genius.com"
CLIENT_ACCESS_TOKEN = token
QUERY_SIZE = 10

In [43]:
# send request and get response in json format.
def _get(path, params=None, headers=None):

    # generate request URL
    requrl = '/'.join([BASE_URL, path])
    token = "Bearer {}".format(CLIENT_ACCESS_TOKEN)
    if headers:
        headers['Authorization'] = token
    else:
        headers = {"Authorization": token}

    response = requests.get(url=requrl, params=params, headers=headers)
    response.raise_for_status()

    return response.json()

def get_artist_songs(artist_id):
    # initialize variables & a list.
    current_page = 1
    next_page = True
    songs = []

    # main loop
    while next_page:

        path = "artists/{}/songs/".format(artist_id)
        params = {'page': current_page}
        data = _get(path=path, params=params)

        page_songs = data['response']['songs']

        if page_songs:
            # add all the songs of current page,
            # and increment current_page value for next loop.
            songs += page_songs
            current_page += 1
        else:
            # if page_songs is empty, quit.
            next_page = False

    # get all the song ids, excluding not-primary-artist songs.
    songs = [song["id"] for song in songs
             if song["primary_artist"]["id"] == artist_id]

    return songs

In [44]:
# read in list of artist names
with open("artists.txt", "r") as f:
    artists = f.readlines()
    
artist_ids = [line.strip().split(',')[0] for line in artists if line.strip().split(',')[0] != 'None']
print(str(len(artists))+" artists total")

0 artists total


In [37]:
# query only a subset of artists at a time to not overwork server
# increment chunk_num on each run
chunk_num = 0 
artists_chunk = artists[QUERY_SIZE*chunk_num:QUERY_SIZE*(chunk_num+1)]

# populate song ids using artist names
for artist_id in artist_ids:

    # get all song ids and make a list.
    song_ids = get_artist_songs(artist_id)

    print("-> " + str(len(song_ids))+" results")

    with open("song_ids.txt", "a") as f:
        for song_id in song_ids:
            f.write(str(artist_id)+","+str(song_id)+"\n")
            
    sleep(.25)

NameError: name 'QUERY_SIZE' is not defined

In [26]:
count=0
with open("song_ids.txt", "r") as f:
    for line in f:
        count+=1
count

7586